#  Extracts PDF metadata to a library catalog

## 0. Installs

In [ ]:
# %pip install --upgrade pip
%pip install -r requirements.txt
# %pip install ipython
%pip install pypdf
%pip install bs4
%pip install requests
#%pip install openpyxl
%pip install tabulate

## 1. Imports

In [1]:
from pypdf import PdfReader
import os
import sys
import logging
import uuid
import pandas as pd
from datetime import datetime, timedelta

In [2]:
'''This litle code block is used anytime you want to import a local module from within a Jupyter Notebook. This is required becuase Jupyter treats each cell as a module.'''

# Navigate up one level from the current notebook's directory to reach the root directory
current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [3]:
import utils

2024-02-16 19:18:36.135 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


found the following file(s) ['library_catalog_2024-02-17T0001Z.xlsx', 'library_catalog_2024-02-16T2346Z.xlsx', 'library_catalog_2024-02-16T2310Z.xlsx', 'library_catalog_2024-02-16T2300Z.xlsx', 'library_catalog_2024-02-16T2236Z.xlsx', 'library_catalog_2024-01-31T0000Z.xlsx', 'library_catalog_2024-01-30T2033Z.xlsx', 'library_catalog_2023-12-19T1900Z.xlsx']


## 2. Set Configurations

In [4]:
pdf_source_directory = "/Users/drew_wilkins/Drews_Files/Drew/Python/ASK/local/PDFs/original_library_documents/CG_General"
library_catalog_directory = "../docs/library_catalog/"
zulu_format = '%Y-%m-%dT%H:%MZ'
leadership_scope = "1_National"
curator = "Wilkins,CA"

## 3. Create the metadata dataframe

In [5]:
# Code to create metadata dictionary

def check_dir_exists(pdf_source_directory):
    '''Check if the path exists'''
    if not os.path.exists(pdf_source_directory):
        print(f"Error: The path '{pdf_source_directory}' does not exist!")
        raise ValueError(f"The path '{pdf_source_directory}' does not exist!")


def check_pdf_for_issues(pdf_path):
    '''
    Check if the PDF has issues like being encrypted or corrupt. 

    If it's encrypted, it will attempt to decrypt using the provided 
    password (in this case, an empty string). If it's corrupt or if 
    there's any other issue, it will log the problem and skip the file.  
    '''
    try:
        pdf = PdfReader(pdf_path)
        if pdf.is_encrypted:
            print(f"Encryption detected for {pdf_path}")
            pdf.decrypt("")
            print(f"All pages accessed: {len(pdf.pages)}")
        return True
    except Exception as e:
        print(f"Issue with {pdf_path}: {e}")
        return False


def remove_multiple_extensions(file_name):
    while os.path.splitext(file_name)[1]:
        file_name = os.path.splitext(file_name)[0]
    return file_name


def get_pdf_metadata(pdf_path):
    '''Extract all metadata fields present in the PDF file 
    along with page count and the hash into a dictionary.'''
    with open(pdf_path, 'rb') as f:
        reader = PdfReader(f)
        metadata = reader.metadata
        file_name = remove_multiple_extensions(os.path.basename(pdf_path))
        CreationDate = metadata.get('/CreationDate', '')[2:16]
        creation_date = datetime.strptime(
            CreationDate, '%Y%m%d%H%M%S') if CreationDate else datetime.utcnow()
        expiration_date = creation_date + timedelta(days=365.25 * 10)
        creation_date = creation_date.strftime(zulu_format)
        return {
            'title': metadata.get('/Title', file_name.replace('_', ' ')),
            'leadership_scope': leadership_scope,
            'page_count': len(reader.pages),
            'creation_date': creation_date,
            'effective_date': creation_date,
            'tagged_date': datetime.utcnow().strftime(zulu_format),
            'upsert_date': datetime.utcnow().strftime(zulu_format),
            'expiration_date': expiration_date.strftime(zulu_format),
            'lifecycle': "",  # set during hand edit
            'aux_specific': True,
            'public_release': True,
            'publication_number': file_name.replace('_', ' '),
            'source': None,
            'organization': None,
            'curator': curator,
            'document_id': utils.compute_doc_id(pdf_path),
            'file_name': file_name,
            # 'pdf_path': pdf_path,
        }

In [6]:
def make_metadata_dict_from_pdfs():
    '''pulls the metadata from all the pdfs into a dataframe with standard formatting
        pdfs in rows and metadata atributes in columns
    '''
    check_dir_exists(pdf_source_directory)

    all_pdfs_metadata = {}

    for root, dirs, files in os.walk(pdf_source_directory):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root, file)
                check_pdf_for_issues(pdf_path)
                utils.compute_doc_id(pdf_path)
                pdf_metadata_dict = get_pdf_metadata(pdf_path)
                all_pdfs_metadata[file] = pdf_metadata_dict

    return all_pdfs_metadata


all_pdfs_metadata = make_metadata_dict_from_pdfs()
print(all_pdfs_metadata)

Encryption detected for /Users/drew_wilkins/Drews_Files/Drew/Python/ASK/local/PDFs/original_library_documents/CG_General/USCG Incident Mgmt Handbook COMDTPUB_P3120.17B_2014-05-01.pdf
All pages accessed: 382
{'USCG Incident Mgmt Handbook COMDTPUB_P3120.17B_2014-05-01.pdf': {'title': 'USCG Incident Mgmt Handbook COMDTPUB P3120', 'leadership_scope': '1_National', 'page_count': 382, 'creation_date': '2014-05-22T11:45Z', 'effective_date': '2014-05-22T11:45Z', 'tagged_date': '2024-02-17T00:18Z', 'upsert_date': '2024-02-17T00:18Z', 'expiration_date': '2024-05-21T23:45Z', 'lifecycle': '', 'aux_specific': True, 'public_release': True, 'publication_number': 'USCG Incident Mgmt Handbook COMDTPUB P3120', 'source': None, 'organization': None, 'curator': 'Wilkins,CA', 'document_id': UUID('2dbb7978-0b08-51d1-8059-98672e2ce535'), 'file_name': 'USCG Incident Mgmt Handbook COMDTPUB_P3120'}, 'Risk Management_CI_3500_3A_03-05-2018.PDF': {'title': 'RISK MANAGEMENT (RM), COMDTINST 3500.3A', 'leadership_scop

In [7]:
# pull the metadata from all the pdfs into a dataframe

new_metadata_df = pd.DataFrame(all_pdfs_metadata).transpose()
print(
    f"Dataframe created with {new_metadata_df.shape[0]} rows, {new_metadata_df.shape[1]} columns")

Dataframe created with 19 rows, 17 columns


## 4. Append New PDF Metadata to Library Catalog and save to a New File

In [8]:
def make_xlsx(df: pd.DataFrame, file_name, directory_name) -> None:
    """A generic function that writes a dataframe to a new Excel file."""

    now_utc = datetime.utcnow()
    # inside function to prevent accidental file overwrites
    timestamp = now_utc.strftime('%Y-%m-%dT%H%MZ')
    file_path = f'{directory_name}{file_name}{timestamp}.xlsx'

    # Save DataFrame to Excel. Index=True metadata_keys as row 1
    # if Index =True then be sure to pd.read_excel( , index_col=0) when you bring it back in
    df.to_excel(file_path, index=False)
    print(f"""Successfully exported:  {file_path}""")

In [9]:
catalog_file_path, last_update_date = utils.get_most_recent_filepath_and_date(
    "library_catalog", library_catalog_directory, "xlsx")

try:
    most_recent_catalog_df = pd.read_excel(catalog_file_path)
    print(
        f"Successfully imported most recent library catalog:  {catalog_file_path}")
    print(
        f"""Dataframe created with {most_recent_catalog_df.shape[0]} rows, {most_recent_catalog_df.shape[1]} columns""")
except Exception as e:
    os.write(
        1, f"Failed to read the most recent library catalog file: {e}\n".encode())
    os.write(1, f"Cannot append so saving as new catalog file: {e}\n".encode())
    make_xlsx(new_metadata_df, "library_catalog", library_catalog_directory)

found the following file(s) ['library_catalog_2024-02-17T0001Z.xlsx', 'library_catalog_2024-02-16T2346Z.xlsx', 'library_catalog_2024-02-16T2310Z.xlsx', 'library_catalog_2024-02-16T2300Z.xlsx', 'library_catalog_2024-02-16T2236Z.xlsx', 'library_catalog_2024-01-31T0000Z.xlsx', 'library_catalog_2024-01-30T2033Z.xlsx', 'library_catalog_2023-12-19T1900Z.xlsx']
Successfully imported most recent library catalog:  ../docs/library_catalog/library_catalog_2024-02-17T0001Z.xlsx
Dataframe created with 217 rows, 17 columns


In [10]:
def append_dataframes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    if df1.columns.equals(df2.columns):
        return pd.concat([df1, df2], ignore_index=True)
    else:
        raise ValueError("DataFrames do not have the same columns")


def check_for_duplicates(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    duplicates = df.duplicated(subset=column_name, keep=False)
    return df[duplicates]


def compare_dfs(df1, df2):
    '''a utility to find redundant PDFs, if needed

    usage
        compare_dfs(new_metadata_df, most_recent_catalog_df)
    '''

    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)
    unique_to_df1 = columns_df1.difference(columns_df2)
    print(f"Columns unique to the first DataFrame: {unique_to_df1}")
    unique_to_df2 = columns_df2.difference(columns_df1)
    print(f"Columns unique to the second DataFrame: {unique_to_df2}")


def append_new_metadata_check_and_export(most_recent_catalog_df, new_metadata_df, directory, file_name):
    try:
        new_catalog_df = append_dataframes(
            most_recent_catalog_df, new_metadata_df)
        duplicate_rows = check_for_duplicates(new_catalog_df, 'document_id')

        if not duplicate_rows.empty:
            logging.warning(
                "Duplicate document IDs found. Use compare_dfs utility function to find and remove")
            logging.info(duplicate_rows)

        make_xlsx(new_catalog_df, file_name, directory)
        os.write(
            1, "Sucessfully appended data to new file. \nAdd/edit the metadata of the appended rows befor ingesting.\n".encode())
        return new_catalog_df

    except ValueError as e:
        logging.error(e)
        return None


append_new_metadata_check_and_export(
    most_recent_catalog_df, new_metadata_df, library_catalog_directory, "library_catalog_")

Successfully exported:  ../docs/library_catalog/library_catalog_2024-02-17T0018Z.xlsx
Sucessfully appended data to new file. 
Add/edit the metadata of the appended rows befor ingesting.


,title,leadership_scope,page_count,creation_date,effective_date,tagged_date,upsert_date,expiration_date,lifecycle,aux_specific,public_release,publication_number,source,organization,curator,document_id,file_name
0,002 24 2024 NATIONAL WORKSHOPS,1_National,3,2024-01-25T07:36Z,2024-01-25T07:36Z,2024-02-16T22:35Z,2024-02-16T22:35Z,2034-01-24T19:36Z,NaN,True,True,ALAUX_002_24,cgaux.org,NaN,"Wilkins,CA",7ce1b8d9-7458-5b30-bff1-8a18067b6794,002_24_2024_NATIONAL_WORKSHOPS
1,006 13 CAMPAIGN TO ELIMINATE SEXUAL ASSAULT ...,1_National,1,2019-11-12T14:27Z,2019-11-12T14:27Z,2024-02-16T22:35Z,2024-02-16T22:35Z,2029-11-12T02:27Z,NaN,True,True,ALAUX_006_13,cgaux.org,NaN,"Wilkins,CA",b15229c0-03ca-5ba3-8466-9c06c0535493,006_13_CAMPAIGN_TO_ELIMINATE_SEXUAL_ASSAULT___...
2,007 13 ALCOAST 253 13 AUXILIARY FOOD SERVICE...,1_National,1,2019-11-12T14:29Z,2019-11-12T14:29Z,2024-02-16T22:35Z,2024-02-16T22:35Z,2029-11-12T02:29Z,NaN,True,True,ALAUX_007_13,cgaux.org,NaN,"Wilkins,CA",a6163904-c1cf-5325-96f8-5dad02b55f12,007_13_ALCOAST_253_13___AUXILIARY_FOOD_SERVICE...
3,01 14 CHIEF DIRECTOR FINAL ACTION ON NATIONAL ...,1_National,1,2019-11-12T10:39Z,2019-11-12T10:39Z,2024-02-16T22:35Z,2024-02-16T22:35Z,2029-11-11T22:39Z,NaN,True,True,ALAUX_01_14,cgaux.org,NaN,"Wilkins,CA",0e3e11bf-dc90-563f-8f15-f7f52b1da0a3,01_14_CHIEF_DIRECTOR_FINAL_ACTION_ON_NATIONAL_...
4,01 16 AUXILIARY MUSIC AUXMU SOP 31JAN2016,1_National,1,2019-10-21T12:25Z,2019-10-21T12:25Z,2024-02-16T22:35Z,2024-02-16T22:35Z,2029-10-21T00:25Z,NaN,True,True,ALAUX_01_16,cgaux.org,NaN,"Wilkins,CA",5f5e0342-24b5-5d92-82fe-0a9adb731c86,01_16_AUXILIARY_MUSIC__AUXMU__SOP___31JAN2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,BCH1B.pdf,1_National,293,2022-11-02T12:05Z,2022-11-02T12:05Z,2024-02-17T00:18Z,2024-02-17T00:18Z,2032-11-02T00:05Z,,True,True,BoatCrewHandbook Boat Operations BCH16114,None,None,"Wilkins,CA",d7118900-79e3-5aa8-bf19-2e0612543c5b,BoatCrewHandbook_Boat Operations BCH16114
232,bch3A.pdf,1_National,219,2022-11-02T08:12Z,2022-11-02T08:12Z,2024-02-17T00:18Z,2024-02-17T00:18Z,2032-11-01T20:12Z,,True,True,BoatCrewHandbook Navigation and Piloting BCH16114,None,None,"Wilkins,CA",7a025fa3-6ce6-5e41-975d-beab92758b8e,BoatCrewHandbook_Navigation and Piloting BCH16114
233,THE U.S. COAST GUARD ADDENDUM TO THE UNITED ST...,1_National,697,2022-10-28T16:24Z,2022-10-28T16:24Z,2024-02-17T00:18Z,2024-02-17T00:18Z,2032-10-28T04:24Z,,True,True,USCG Addendum to US NSS to IAMSAR CI 16130 2G ...,None,None,"Wilkins,CA",cb5f31b9-06fb-55bd-ab2c-2d8db39ea4f2,USCG Addendum to US NSS to IAMSAR CI_16130_2G_...
234,bch4Aupdated.pdf,1_National,312,2022-11-02T08:14Z,2022-11-02T08:14Z,2024-02-17T00:18Z,2024-02-17T00:18Z,2032-11-01T20:14Z,,True,True,BoatCrewHandbook Seamanship Fundamentals BCH16114,None,None,"Wilkins,CA",ee9fc91f-4a22-5cc0-8054-91135a569fce,BoatCrewHandbook_Seamanship Fundamentals BCH16114
